### 데이터 셋업

In [ ]:
# ======1단계: 입력 폴더 경로 설정
path_in = r"..\assets\ply\impeller\3wings"   # Windows 경로 예시
downsample_N = 20000         # 각각 포인트 샘플링 수


In [ ]:
import os
import numpy as np
import open3d as o3d
from pathlib import Path

#======= 2단계: 단일 PLY 파일 → 점군으로 변환 후 .xyz 저장 =======
def convert_to_xyz(path_in: str, downsample_N: int = 20000):
    """
    단일 PLY 파일 → 점군으로 변환 후 .xyz 저장
    """
    mesh = o3d.io.read_triangle_mesh(path_in)
    if mesh.has_triangles():
        pcd = mesh.sample_points_poisson_disk(number_of_points=downsample_N)
    else:
        pcd = o3d.io.read_point_cloud(path_in)
        if len(pcd.points) > downsample_N:
            pts = np.asarray(pcd.points)
            idx = np.random.choice(len(pts), downsample_N, replace=False)
            pcd.points = o3d.utility.Vector3dVector(pts[idx])

    out_path = Path(path_in).with_name(Path(path_in).stem + "_point.xyz")
    np.savetxt(out_path, np.asarray(pcd.points), fmt="%.6f")
    print(f"[OK] {path_in} → {out_path} (N={len(pcd.points)})")


# 지정한 폴더에 있는 모든 .ply 파일 변환
folder = Path(path_in)
if not folder.is_dir():
    print(f"[ERR] 폴더 경로가 아님: {folder}")
else:
    ply_files = list(folder.glob("*.ply"))
    if not ply_files:
        print(f"[INFO] {folder} 안에 .ply 파일이 없습니다.")
    else:
        for ply in ply_files:
            convert_to_xyz(str(ply), downsample_N=downsample_N)
        print(f"[DONE] 총 {len(ply_files)} 개 파일 변환 완료.")


In [ ]:
import numpy as np
import re
from pathlib import Path

# ====== 3단계: .xyz 파일 → .xyzc 파일로 변환 ======
def convert_xyz_to_xyzc(xyz_path: Path):
    """
    파일명에서 숫자 추출 → 라벨로 추가 → .xyzc 저장
    예: impeller_b3_010_point.xyz → 라벨 = 10.00000000
    """
    # 파일 이름에서 숫자 추출
    m = re.search(r"_(\d+)", xyz_path.stem)
    if not m:
        print(f"[SKIP] 라벨 숫자 추출 실패: {xyz_path.name}")
        return None
    
    label_value = float(m.group(1))  # "010" → 10.0

    # xyz 불러오기
    xyz = np.loadtxt(xyz_path)
    if xyz.ndim == 1:
        xyz = xyz.reshape(-1, 3)

    # 라벨 추가
    label_column = np.full((xyz.shape[0], 1), label_value)
    xyzc = np.hstack((xyz, label_column))

    # 저장
    out_path = xyz_path.with_suffix(".xyzc")
    np.savetxt(out_path, xyzc, fmt="%.6f %.6f %.6f %.8f")
    print(f"[OK] {xyz_path.name} → {out_path.name} (label={label_value:.8f})")
    return out_path


# ====== 폴더 내 모든 _point.xyz 파일 처리 ======
folder = Path(path_in)
xyz_files = list(folder.glob("*_point.xyz"))

if not xyz_files:
    print(f"[INFO] {folder} 안에 '_point.xyz' 파일이 없습니다.")
else:
    for xyz_file in xyz_files:
        convert_xyz_to_xyzc(xyz_file)

    print(f"[DONE] 총 {len(xyz_files)} 개 파일 변환 완료.")


In [ ]:
import numpy as np
from pathlib import Path

# ====== 4단계: 모든 XYZC 파일 병합 ======

output_file = Path(path_in) / "merged_all.xyzc"  # 최종 합쳐진 파일명

# 폴더 내 모든 .xyzc 찾기
folder = Path(path_in)
xyzc_files = list(folder.glob("*.xyzc"))

if not xyzc_files:
    print(f"[INFO] {folder} 안에 .xyzc 파일이 없습니다.")
else:
    merged_data = []
    for f in xyzc_files:
        try:
            data = np.loadtxt(f)
            if data.ndim == 1:  # 한 줄짜리 방지
                data = data.reshape(1, -1)
            merged_data.append(data)
            print(f"[READ] {f.name} (N={len(data)})")
        except Exception as e:
            print(f"[ERR] {f.name} 불러오기 실패: {e}")

    if merged_data:
        merged_data = np.vstack(merged_data)
        np.savetxt(output_file, merged_data, fmt="%.6f %.6f %.6f %.8f")
        print(f"[DONE] {len(xyzc_files)} 개 파일 합침 → {output_file} (총 {len(merged_data)} 포인트)")
    else:
        print("[INFO] 합칠 데이터가 없습니다.")


In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt

def load_xyzc(path):
    data = np.loadtxt(path)
    if data.shape[1] != 4:
        raise ValueError("Input file must have 4 columns: x, y, z, label")
    return data

def print_layer_distribution(label_array):
    total = len(label_array)
    label_counts = Counter(label_array)
    print("📊 Layer distribution (by percentage):")
    for label, count in sorted(label_counts.items()):
        percent = 100.0 * count / total
        print(f"  Layer {label:.6f}: {count} points ({percent:.2f}%)")
    print("")

def visualize_point_cloud(data):
    x, y, z, label = data[:,0], data[:,1], data[:,2], data[:,3]
    print_layer_distribution(label)

    unique_labels = np.unique(label)
    label_to_color = {l: plt.cm.tab20(i / max(len(unique_labels)-1, 1)) 
                      for i, l in enumerate(unique_labels)}
    colors = np.array([label_to_color[l] for l in label])

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z, c=colors, s=1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Point Cloud Visualization by Layer')
    plt.tight_layout()
    plt.show()

# Load and visualize merged point cloud
merged_xyzc_path = output_file  # already defined in your notebook
data = load_xyzc(merged_xyzc_path)
visualize_point_cloud(data)

In [ ]:
# ==== Jupyter 한 셀에서 모두 해결 ====
# 1) 필수 임포트
import os, sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # <- Pylance 'optim' undefined 해결

from scipy.spatial import cKDTree
from scipy.stats import mode

def normalize_points(points):
    # (N,3) → 원점 기준, 단위 스케일
    c = points.mean(0)
    p = points - c
    s = np.max(np.linalg.norm(p, axis=1))
    p = p / (s + 1e-8)
    return p.astype(np.float32), (c, s)

# 인덱스 보존형 증강(라벨 손실 없음): jitter/작은 회전/이동/스케일
def aug_index_preserving(p, jitter_sigma=0.002, jitter_clip=0.01,
                         rot_deg=2, trans_mag=0.005, scale_jitter=0.02):
    q = p.copy()
    # jitter
    n = np.clip(np.random.normal(0, jitter_sigma, q.shape), -jitter_clip, jitter_clip)
    q += n
    # small rotation (z축 예시; 필요시 x/y도 추가)
    th = np.deg2rad(np.random.uniform(-rot_deg, rot_deg))
    c, s = np.cos(th), np.sin(th)
    Rz = np.array([[c,-s,0],[s,c,0],[0,0,1]], np.float32)
    q = (q @ Rz.T).astype(np.float32)
    # small translation
    t = np.random.uniform(-trans_mag, trans_mag, (1,3)).astype(np.float32)
    q = q + t
    # small scaling
    sc = 1.0 + np.random.uniform(-scale_jitter, scale_jitter)
    q = q * sc
    return q

def knn_smooth(points, preds, k=16):
    k = int(min(k, len(points)))
    idx = cKDTree(points).query(points, k=k)[1]
    smoothed = mode(preds[idx], axis=1, keepdims=False).mode
    return np.asarray(smoothed, dtype=preds.dtype)



In [ ]:

SRC_ROOT = os.path.abspath(".")  # 예: 프로젝트 루트
if SRC_ROOT not in sys.path:
    sys.path.append(SRC_ROOT)

try:
    # 보통 파일 경로는 src/PointNet_seg.py 형태
    from src.PointNet_seg import PointNetSeg  # <- Pylance 'PointNetSeg' undefined 해결
except Exception as e:
    raise ImportError(
        "PointNetSeg를 import하지 못했습니다. "
        "노트북 작업 디렉토리 기준으로 'src/PointNet_seg.py'가 있는지 확인하고, "
        "경로가 다르면 SRC_ROOT를 맞춰주세요."
    ) from e
# 3) FocalLoss 정의 (학습 스크립트와 동일/호환)
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction
        self.ce = nn.CrossEntropyLoss(weight=weight, reduction='none')

    def forward(self, logits, targets):
        # logits: (B,C) 또는 (N,C) 형태, targets: (B,) 또는 (N,)
        ce = self.ce(logits, targets)             # (N,)
        pt = torch.exp(-ce)                       # p = exp(-CE)
        loss = ((1 - pt) ** self.gamma) * ce      # focal
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss

def train_impeller_robust(xyzc_path, model_out, lr, stop_loss, device, sample_size,
                          lambda_consis=0.5, epochs=200):
    print(f"[Train-Robust] Using device: {device}")
    data = np.loadtxt(xyzc_path).astype(np.float32)
    points = data[:, :3]
    orig_labels = data[:, 3].astype(np.int64)

    # 0) 정규화
    points, _ = normalize_points(points)

    # 1) 라벨 매핑
    unique_labels = np.unique(orig_labels)
    label_to_idx = {lbl: idx for idx, lbl in enumerate(unique_labels)}
    labels = np.array([label_to_idx[l] for l in orig_labels], dtype=np.int64)

    # 2) 모델/손실/옵티마이저
    num_classes = len(unique_labels)
    model = PointNetSeg(num_classes=num_classes).to(device)

    # class-imbalance 가중치(있으면 그대로 사용)
    counts = np.bincount(labels, minlength=num_classes)
    inv_freq = 1.0 / (counts + 1e-6)
    weights = inv_freq / np.sum(inv_freq) * num_classes
    class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

    # Focal + Consistency
    criterion_sup = FocalLoss(gamma=2.0, weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

    best_loss = float('inf'); best_state = None; full_loss_history = []
    N = points.shape[0]; steps_per_epoch = max(1, N // sample_size)
    model.train()

    for epoch in range(1, epochs+1):
        cycle_loss = 0.0
        # 점진적 램프업(초반엔 감독 위주, 후반엔 일관성 비중 ↑)
        lam = lambda_consis * min(1.0, epoch / (epochs*0.3))

        for _ in range(steps_per_epoch):
            idx = np.random.choice(N, sample_size, replace=False)
            x_clean = points[idx]          # (B,3)
            y_batch = labels[idx]          # (B,)

            # 인덱스 유지하는 증강만 사용(라벨 매칭 깨지지 않도록)
            x_aug = aug_index_preserving(x_clean)

            x_clean_t = torch.from_numpy(x_clean.T).unsqueeze(0).to(device)  # (1,3,B)
            x_aug_t   = torch.from_numpy(x_aug.T).unsqueeze(0).to(device)
            y_t       = torch.from_numpy(y_batch).to(device)

            optimizer.zero_grad()
            logits_clean = model(x_clean_t).squeeze(0).permute(1,0)  # (B,C)
            logits_aug   = model(x_aug_t).squeeze(0).permute(1,0)    # (B,C)

            # train 루프 내부
            loss_sup = criterion_sup(logits_clean, y_t)

            beta = 0.2  # 0.1~0.3 권장
            loss_sup_aug = criterion_sup(logits_aug, y_t)

            p_clean = F.softmax(logits_clean, dim=1)
            p_aug   = F.softmax(logits_aug,   dim=1)
            loss_cons = F.mse_loss(p_clean, p_aug)

            loss = loss_sup + beta*loss_sup_aug + lam*loss_cons

            loss.backward(); optimizer.step()
            cycle_loss += float(loss.item())

        avg_loss = cycle_loss / steps_per_epoch
        scheduler.step()
        print(f"Epoch {epoch}/{epochs}  Loss: {avg_loss:.6f}  (sup={loss_sup.item():.6f}, cons={loss_cons.item():.6f}, lam={lam:.3f})")

        # 10epoch마다 full-loss(정규화된 전체 포인트 기준)
        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                x_full = torch.from_numpy(points.T).unsqueeze(0).to(device)
                y_full = torch.from_numpy(labels).to(device)
                logits_full = model(x_full).squeeze(0).permute(1,0)
                full_loss = criterion_sup(logits_full, y_full).item()
            model.train()
            full_loss_history.append(full_loss)
            print(f"  FullLoss: {full_loss:.6f}")

            if full_loss < best_loss:
                best_loss = full_loss; best_state = model.state_dict()
                save_dir = os.path.dirname(model_out) or "."
                os.makedirs(save_dir, exist_ok=True)
                torch.save({'state_dict': best_state, 'unique_labels': unique_labels.tolist()},
                           os.path.join(save_dir, 'best_'+os.path.basename(model_out)))
                print(f"  [Best] FullLoss improved → {best_loss:.6f}")

            # 간단 early stop (원하면 검증 셋 분리 추천)
            if len(full_loss_history)>=5 and all(l <= stop_loss for l in full_loss_history[-5:]):
                print(f"  [EarlyStop] last 5 FullLoss ≤ {stop_loss}")
                break

    # 최종 저장
    os.makedirs(os.path.dirname(model_out), exist_ok=True)
    final_state = best_state if best_state is not None else model.state_dict()
    torch.save({'state_dict': final_state,
                'unique_labels': unique_labels.tolist()}, model_out)
    print(f"[Saved] Model → {model_out}  (best_loss={best_loss:.6f})")


In [ ]:
def infer_impeller_robust(xyz_path, model_path, device, tta=5, knn_k=16,
                          small_jitter=0.001, small_rot_deg=2, small_trans=0.003):
    print(f"[Infer-Robust] Using {device}")
    data = np.loadtxt(xyz_path).astype(np.float32)
    orig_points = data[:, :3]                         # 원본 보관
    points, (c, s) = normalize_points(orig_points)    # 정규화

    

    # 모델 로드
    chk = torch.load(model_path, map_location=device)
    inv_map = {i:l for i,l in enumerate(chk['unique_labels'])}
    num_classes = len(chk['unique_labels'])
    model = PointNetSeg(num_classes=num_classes).to(device)
    model.load_state_dict(chk['state_dict']); model.eval()

    # TTA: 작은 노이즈/회전/이동 여러 번 → 확률 평균
    with torch.no_grad():
        acc_probs = None
        for _ in range(tta):
            p_aug = aug_index_preserving(points,
                                         jitter_sigma=small_jitter, jitter_clip=3*small_jitter,
                                         rot_deg=small_rot_deg, trans_mag=small_trans, scale_jitter=0.0)
            x = torch.from_numpy(p_aug.T).unsqueeze(0).to(device)
            logits = model(x).squeeze(0)          # (C,N)
            probs = torch.softmax(logits, dim=0)  # (C,N)
            acc_probs = probs if acc_probs is None else (acc_probs + probs)
        probs_avg = acc_probs / float(tta)
        preds = probs_avg.argmax(dim=0).cpu().numpy()

    # KNN 스무딩
    preds = knn_smooth(points, preds, k=knn_k)

    # 저장 (원본 스케일 좌표를 쓰고 싶으면 정규화 이전 좌표 따로 보관하세요)
    out_xyz = points * s + c                          # 복원!
    out = np.hstack([out_xyz, np.array([inv_map[int(i)] for i in preds])[:, None]])
    out_path = xyz_path.replace('.xyz','_impeller_pred.xyzc')
    np.savetxt(out_path, out, fmt='%.6f'); print(f"Saved → {out_path}")
    return out_path


In [ ]:
# Train
train_impeller_robust(
    xyzc_path="../assets/ply/impeller/3wings/impeller_b3_merged_all.xyzc",
    model_out="./models/impeller_seg_robust_b3.pth",
    lr=1e-4, stop_loss=0.001,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    sample_size=2000,
    lambda_consis=0.2, epochs=2000
)

# Infer
infer_impeller_robust(
    xyz_path="C:\\Users\\user\\Documents\\GitHub\\Ai_coding_study\\point2cad\\assets\\xyz\\Impeller_b3_gpu.xyz",
    model_path="./models/impeller_seg_robust_b3.pth",
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    tta=5, knn_k=16
)



In [ ]:
# ==== Jupyter 한 셀에서 모두 해결 (Robust PointNetSeg) ====
# 필수 임포트
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from scipy.spatial import cKDTree
from scipy.stats import mode

# ------------------------------------------------------------
# 유틸
# ------------------------------------------------------------
def normalize_points(points: np.ndarray):
    """(N,3) → 원점 기준 단위 스케일 정규화, (정규화된 점, (중심, 스케일)) 반환"""
    c = points.mean(0)
    p = points - c
    s = np.max(np.linalg.norm(p, axis=1))
    p = p / (s + 1e-8)
    return p.astype(np.float32), (c, s)

def aug_index_preserving(p: np.ndarray,
                         jitter_sigma=0.002, jitter_clip=0.01,
                         rot_deg=2.0, trans_mag=0.005, scale_jitter=0.02):
    """인덱스 보존형 증강(라벨 손실 없음)"""
    q = p.copy()
    # jitter
    n = np.clip(np.random.normal(0, jitter_sigma, q.shape), -jitter_clip, jitter_clip)
    q += n
    # small rotation (z축, 필요 시 x/y도 추가 가능)
    th = np.deg2rad(np.random.uniform(-rot_deg, rot_deg))
    c, s = np.cos(th), np.sin(th)
    Rz = np.array([[c,-s,0],[s,c,0],[0,0,1]], np.float32)
    q = (q @ Rz.T).astype(np.float32)
    # small translation
    t = np.random.uniform(-trans_mag, trans_mag, (1,3)).astype(np.float32)
    q = q + t
    # small scaling
    sc = 1.0 + np.random.uniform(-scale_jitter, scale_jitter)
    q = q * sc
    return q

def knn_smooth(points: np.ndarray, preds: np.ndarray, k=16):
    """KNN 다수결 라벨 스무딩"""
    k = int(min(k, len(points)))
    idx = cKDTree(points).query(points, k=k)[1]
    smoothed = mode(preds[idx], axis=1, keepdims=False).mode
    return np.asarray(smoothed, dtype=preds.dtype)

# ------------------------------------------------------------
# 모델 임포트
# ------------------------------------------------------------
SRC_ROOT = os.path.abspath(".")  # 예: 프로젝트 루트
if SRC_ROOT not in sys.path:
    sys.path.append(SRC_ROOT)

try:
    # 보통 파일 경로는 src/PointNet_seg.py 형태
    from src.PointNet_seg import PointNetSeg
except Exception as e:
    raise ImportError(
        "PointNetSeg를 import하지 못했습니다. "
        "노트북 작업 디렉토리 기준으로 'src/PointNet_seg.py'가 있는지 확인하고, "
        "경로가 다르면 SRC_ROOT를 맞춰주세요."
    ) from e

# ------------------------------------------------------------
# 손실 정의
# ------------------------------------------------------------
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction
        self.ce = nn.CrossEntropyLoss(weight=weight, reduction='none')

    def forward(self, logits, targets):
        # logits: (N,C), targets: (N,)
        ce = self.ce(logits, targets)             # (N,)
        pt = torch.exp(-ce)                       # p = exp(-CE)
        loss = ((1 - pt) ** self.gamma) * ce      # focal
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        return loss

# ------------------------------------------------------------
# (옵션) EMA Teacher & KL 일관성
# ------------------------------------------------------------
import copy
def create_ema(model):
    ema = copy.deepcopy(model).eval()
    for p in ema.parameters():
        p.requires_grad_(False)
    return ema

@torch.no_grad()
def update_ema(student, teacher, mu=0.999):
    for ps, pt in zip(student.parameters(), teacher.parameters()):
        pt.data.mul_(mu).add_(ps.data, alpha=1-mu)

# ------------------------------------------------------------
# 학습 루틴
# ------------------------------------------------------------
def train_impeller_robust(xyzc_path,
                          model_out="./models/impeller_seg_robust.pth",
                          lr=1e-4, stop_loss=0.001,
                          device=None,
                          sample_size=2000,
                          lambda_consis=0.2,   # 일관성 손실 가중치(최대)
                          beta_aug=0.2,        # 증강 배치 감독 가중치
                          epochs=2000,
                          use_label_smoothing=False, label_smoothing=0.05,
                          use_ema_kld=False, ema_mu=0.999, cons_temp=2.0,
                          save_curve=True):
    """
    - 증강 배치에도 감독 손실 추가(beta_aug)
    - 일관성 손실 램프업(lambda_consis * min(1, epoch/(epochs*0.3)))
    - (옵션) EMA teacher + KL 일관성
    - CosineAnnealingLR 스케줄
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"[Train-Robust] Using device: {device}")

    # 데이터 로드
    data = np.loadtxt(xyzc_path).astype(np.float32)
    points = data[:, :3]
    orig_labels = data[:, 3].astype(np.int64)

    # 정규화
    points, _ = normalize_points(points)

    # 라벨 인덱스화
    unique_labels = np.unique(orig_labels)
    label_to_idx = {lbl: idx for idx, lbl in enumerate(unique_labels)}
    labels = np.array([label_to_idx[l] for l in orig_labels], dtype=np.int64)

    # 모델/손실/옵티마이저
    num_classes = len(unique_labels)
    model = PointNetSeg(num_classes=num_classes).to(device)

    # 클래스 불균형 가중치
    counts = np.bincount(labels, minlength=num_classes)
    inv_freq = 1.0 / (counts + 1e-6)
    weights = inv_freq / np.sum(inv_freq) * num_classes
    class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

    # 감독 손실 선택
    if use_label_smoothing:
        criterion_sup = nn.CrossEntropyLoss(weight=class_weights,
                                            label_smoothing=label_smoothing)
    else:
        criterion_sup = FocalLoss(gamma=2.0, weight=class_weights)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

    # (옵션) EMA teacher
    ema_model = create_ema(model) if use_ema_kld else None

    best_loss = float('inf')
    best_state = None
    full_loss_history = []

    N = points.shape[0]
    steps_per_epoch = max(1, int(np.ceil(N / max(1, sample_size))))

    model.train()
    for epoch in range(1, epochs + 1):
        cycle_loss = 0.0
        # 일관성 램프업 (초반 30% 구간)
        lam = lambda_consis * min(1.0, epoch / (epochs * 0.3))

        # (선택) 후반부 약간 완화
        if epoch > 0.8 * epochs:
            lam *= 0.5

        # 증강 강도도 후반에 살짝 감소
        cur_sigma = float(np.interp(epoch, [1, 0.7*epochs, epochs],
                                    [0.002, 0.002, 0.001]))
        cur_rot   = float(np.interp(epoch, [1, 0.7*epochs, epochs],
                                    [2.0,   2.0,   1.0  ]))
        cur_trans = 0.003  # 고정 (필요시 스케줄링)

        for _ in range(steps_per_epoch):
            # 샘플링 (데이터가 sample_size보다 적으면 replace=True)
            replace_flag = sample_size > N
            idx = np.random.choice(N, sample_size, replace=replace_flag)
            x_clean = points[idx]      # (B,3)
            y_batch = labels[idx]      # (B,)

            # 인덱스 보존 증강
            x_aug = aug_index_preserving(
                x_clean,
                jitter_sigma=cur_sigma, jitter_clip=3*cur_sigma,
                rot_deg=cur_rot, trans_mag=cur_trans, scale_jitter=0.01
            )

            x_clean_t = torch.from_numpy(x_clean.T).unsqueeze(0).to(device)  # (1,3,B)
            x_aug_t   = torch.from_numpy(x_aug.T).unsqueeze(0).to(device)
            y_t       = torch.from_numpy(y_batch).to(device)

            optimizer.zero_grad()

            # forward
            logits_clean = model(x_clean_t).squeeze(0).permute(1, 0)  # (B,C)
            logits_aug   = model(x_aug_t).squeeze(0).permute(1, 0)    # (B,C)

            # 감독 손실 (clean)
            loss_sup = criterion_sup(logits_clean, y_t)

            # 증강 배치 감독(약하게)
            loss_sup_aug = criterion_sup(logits_aug, y_t) if beta_aug > 0 else 0.0

            # 일관성 손실
            if use_ema_kld:
                # EMA teacher KL( student||teacher )
                with torch.no_grad():
                    t_logits = ema_model(x_aug_t).squeeze(0).permute(1,0) / cons_temp
                    t_prob   = F.softmax(t_logits, dim=1)
                s_logits = logits_aug / cons_temp
                log_prob = F.log_softmax(s_logits, dim=1)
                loss_cons = F.kl_div(log_prob, t_prob, reduction='batchmean') * (cons_temp * cons_temp)
            else:
                # 단순 MSE(consistency)
                p_clean = F.softmax(logits_clean, dim=1)
                p_aug   = F.softmax(logits_aug,   dim=1)
                loss_cons = F.mse_loss(p_clean, p_aug)

            # 총손실
            if isinstance(loss_sup_aug, float):
                loss = loss_sup + lam * loss_cons
            else:
                loss = loss_sup + beta_aug * loss_sup_aug + lam * loss_cons

            loss.backward()
            optimizer.step()

            # EMA 업데이트
            if ema_model is not None:
                update_ema(model, ema_model, mu=ema_mu)

            cycle_loss += float(loss.item())

        avg_loss = cycle_loss / steps_per_epoch
        scheduler.step()
        # 모니터링: 마지막 스텝의 loss 항들 출력 (대표값)
        cons_val = float(loss_cons.detach().cpu()) if torch.is_tensor(loss_cons) else float(loss_cons)
        print(f"Epoch {epoch}/{epochs}  Loss: {avg_loss:.6f}  (sup={float(loss_sup.detach().cpu()):.6f}, cons={cons_val:.6f}, lam={lam:.3f})")

        # 10 epoch마다 전체 평가(정규화 좌표 기준)
        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                x_full = torch.from_numpy(points.T).unsqueeze(0).to(device)
                y_full = torch.from_numpy(labels).to(device)
                logits_full = model(x_full).squeeze(0).permute(1,0)
                full_loss = criterion_sup(logits_full, y_full).item()
            model.train()

            full_loss_history.append(full_loss)
            print(f"  FullLoss: {full_loss:.6f}")

            # best 저장
            save_dir = os.path.dirname(model_out) or "."
            os.makedirs(save_dir, exist_ok=True)
            if full_loss < best_loss:
                best_loss = full_loss
                best_state = model.state_dict()
                torch.save({'state_dict': best_state, 'unique_labels': unique_labels.tolist()},
                           os.path.join(save_dir, 'best_'+os.path.basename(model_out)))
                print(f"  [Best] FullLoss improved → {best_loss:.6f}")

            # 간단 early stop
            if len(full_loss_history) >= 5 and all(l <= stop_loss for l in full_loss_history[-5:]):
                print(f"  [EarlyStop] last 5 FullLoss ≤ {stop_loss}")
                break

    # 최종 저장
    os.makedirs(os.path.dirname(model_out) or ".", exist_ok=True)
    final_state = best_state if best_state is not None else model.state_dict()
    torch.save({'state_dict': final_state, 'unique_labels': unique_labels.tolist()}, model_out)
    print(f"[Saved] Model → {model_out}  (best_loss={best_loss:.6f})")

    # Loss curve 저장(옵션)
    if save_curve and len(full_loss_history) > 0:
        plt.figure(figsize=(6,4))
        plt.plot(full_loss_history, marker='o')
        plt.title('Full Loss History (every 10 epochs)')
        plt.xlabel('Eval # (x10 epochs)')
        plt.ylabel('Loss')
        plt.grid(True, alpha=0.3)
        curve_path = os.path.join(os.path.dirname(model_out) or ".", "full_loss_curve.png")
        plt.savefig(curve_path, dpi=150, bbox_inches="tight")
        print(f"[Saved] Loss curve → {curve_path}")

# ------------------------------------------------------------
# 추론 루틴
# ------------------------------------------------------------
def infer_impeller_robust(xyz_path, model_path, device=None,
                          tta=5, knn_k=16,
                          small_jitter=0.001, small_rot_deg=2, small_trans=0.003):
    """
    - 입력 xyz를 정규화 → TTA(작은 증강 여러 번) 확률 평균 → KNN 스무딩 → 원본 스케일 복원 → xyzc 저장
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"[Infer-Robust] Using {device}")

    data = np.loadtxt(xyz_path).astype(np.float32)
    orig_points = data[:, :3]
    points, (c, s) = normalize_points(orig_points)  # 정규화 (한 번만)

    # 모델 로드
    chk = torch.load(model_path, map_location=device)
    inv_map = {i: l for i, l in enumerate(chk['unique_labels'])}
    num_classes = len(chk['unique_labels'])
    model = PointNetSeg(num_classes=num_classes).to(device)
    model.load_state_dict(chk['state_dict'])
    model.eval()

    # TTA: 작은 노이즈/회전/이동 여러 번 → 확률 평균
    with torch.no_grad():
        acc_probs = None
        for _ in range(tta):
            p_aug = aug_index_preserving(points,
                                         jitter_sigma=small_jitter, jitter_clip=3*small_jitter,
                                         rot_deg=small_rot_deg, trans_mag=small_trans, scale_jitter=0.0)
            x = torch.from_numpy(p_aug.T).unsqueeze(0).to(device)  # (1,3,N)
            logits = model(x).squeeze(0)          # (C,N)
            probs = torch.softmax(logits, dim=0)  # (C,N)
            acc_probs = probs if acc_probs is None else (acc_probs + probs)
        probs_avg = acc_probs / float(tta)
        preds = probs_avg.argmax(dim=0).cpu().numpy()

    # KNN 스무딩(정규화 좌표에서)
    preds = knn_smooth(points, preds, k=knn_k)

    # 원본 스케일 복원
    out_xyz = points * s + c
    out = np.hstack([out_xyz, np.array([inv_map[int(i)] for i in preds])[:, None]])
    out_path = xyz_path.replace('.xyz', '_impeller_pred.xyzc')
    np.savetxt(out_path, out, fmt='%.6f')
    print(f"[Saved] → {out_path}")
    return out_path

# ------------------------------------------------------------
# 사용 예시 (경로/파라미터는 상황에 맞게 수정)
# ------------------------------------------------------------
if __name__ == "__main__":
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Train
    train_impeller_robust(
        xyzc_path="../assets/ply/impeller/3wings/impeller_b3_merged_all.xyzc",
        model_out="./models/impeller_seg_robust_b3.pth",
        lr=1e-4, stop_loss=0.001,
        device=dev,
        sample_size=2000,
        lambda_consis=0.2,   # 일관성 가중치
        beta_aug=0.2,        # 증강 배치 감독 가중치
        epochs=2000,
        use_label_smoothing=False,  # True로 바꾸면 CE + label smoothing 사용
        label_smoothing=0.05,
        use_ema_kld=False,   # True로 바꾸면 EMA Teacher + KL 일관성 사용
        ema_mu=0.999, cons_temp=2.0,
        save_curve=True
    )

    # Infer
    infer_impeller_robust(
        xyz_path=r"C:\Users\user\Documents\GitHub\Ai_coding_study\point2cad\assets\xyz\Impeller_b3_gpu.xyz",
        model_path="./models/impeller_seg_robust_b3.pth",
        device=dev,
        tta=5, knn_k=16,
        small_jitter=0.001, small_rot_deg=2, small_trans=0.003
    )
